<a href="https://colab.research.google.com/github/vijjus/CV/blob/master/CTC_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [15]:
import torch
import torch.nn as nn

In [2]:
# Target are to be padded
T = 50      # Input sequence length
C = 20      # Number of classes (including blank)
N = 16      # Batch size
S = 30      # Target sequence length of longest target in batch (padding length)
S_min = 10  # Minimum target length, for demonstration purposes

In [19]:
# Initialize random batch of input vectors, for *size = (T,N,C)
input = torch.randn(T, N, C)
input.shape

torch.Size([50, 16, 20])

In [20]:
input

tensor([[[-1.8133e-01,  5.5486e-02, -1.1801e+00,  ...,  5.9338e-01,
          -7.6976e-03, -4.4068e-01],
         [-1.0459e+00,  5.2527e-01,  8.9902e-01,  ...,  2.4780e+00,
           9.7984e-01, -6.1498e-01],
         [ 1.0073e+00, -9.0063e-01, -6.1997e-01,  ...,  1.7867e-01,
           8.3250e-01, -8.6403e-01],
         ...,
         [ 1.1915e+00, -1.8326e+00, -8.7389e-01,  ...,  1.2519e+00,
           2.6438e-02, -3.3346e-01],
         [-3.4131e-01, -2.5401e+00,  1.8421e-01,  ...,  5.2401e-01,
           2.0715e+00, -8.6275e-02],
         [ 4.2230e-01, -1.1417e+00,  2.1390e+00,  ...,  3.2776e-01,
           1.4635e-01, -6.3199e-01]],

        [[-8.9254e-01,  8.2549e-01,  6.1904e-01,  ..., -1.9236e-01,
           1.3014e+00,  3.5321e-01],
         [-3.6969e-01,  1.9278e-01, -5.7103e-01,  ...,  1.8236e+00,
           1.2948e-01,  8.3630e-02],
         [ 1.4288e+00, -1.2932e+00,  9.1205e-01,  ...,  2.7004e-01,
           1.1852e+00, -1.6636e+00],
         ...,
         [ 3.0265e+00, -8

In [21]:
input = input.log_softmax(2)
print(input.shape)

torch.Size([50, 16, 20])


In [22]:
input = input.detach()
input = input.requires_grad_()
input

tensor([[[-4.0006, -3.7638, -4.9993,  ..., -3.2259, -3.8270, -4.2600],
         [-5.1843, -3.6131, -3.2394,  ..., -1.6604, -3.1586, -4.7534],
         [-2.3850, -4.2929, -4.0122,  ..., -3.2136, -2.5598, -4.2563],
         ...,
         [-2.4962, -5.5203, -4.5615,  ..., -2.4358, -3.6612, -4.0211],
         [-4.0986, -6.2974, -3.5731,  ..., -3.2333, -1.6858, -3.8436],
         [-3.1145, -4.6785, -1.3978,  ..., -3.2090, -3.3904, -4.1688]],

        [[-4.5340, -2.8159, -3.0224,  ..., -3.8338, -2.3401, -3.2882],
         [-3.5717, -3.0092, -3.7730,  ..., -1.3783, -3.0725, -3.1183],
         [-2.2086, -4.9306, -2.7254,  ..., -3.3674, -2.4522, -5.3010],
         ...,
         [-0.9715, -4.8235, -4.2978,  ..., -3.8504, -5.7768, -3.2434],
         [-3.2400, -3.7012, -4.2531,  ..., -2.7267, -3.9200, -1.9392],
         [-3.8653, -2.2009, -3.7312,  ..., -3.5750, -2.1122, -2.1276]],

        [[-2.7735, -2.5209, -3.6197,  ..., -3.4913, -5.1966, -3.3136],
         [-2.3972, -2.5922, -4.6198,  ..., -3

In [11]:
# Initialize random batch of targets (0 = blank, 1:C = classes)
target = torch.randint(low=1, high=C, size=(N, S), dtype=torch.long)
print(target.shape)
target

torch.Size([16, 30])


tensor([[17, 11,  6, 15, 14,  8, 15, 18,  8, 19,  2,  1,  4,  2, 18, 19, 18,  9,
          2,  5, 13,  8, 15, 12, 15, 16,  5, 17, 18, 16],
        [ 6,  5,  3, 19, 15,  9, 13, 14, 15, 17, 17,  9,  7, 17, 17, 13,  8, 15,
         12, 10, 11, 11,  2,  8,  8,  9,  2,  6, 19, 12],
        [11, 19,  7,  7, 10, 13,  6, 15, 17,  9,  7, 10,  7, 16, 15,  3,  2, 17,
         17,  7, 19, 11,  9,  2, 15,  6, 11, 19, 17, 10],
        [ 7,  9, 13, 14,  9, 10, 10, 19,  9, 16, 15,  9, 16, 11,  8,  9,  3, 12,
         10,  6,  9, 18,  4, 14, 12,  9, 12,  5,  8, 14],
        [ 8,  5, 17, 15, 15, 17, 14,  7, 11,  2,  6, 17,  7,  8, 13, 10, 16, 15,
          4,  2, 18, 12, 19, 12,  9,  5, 19, 14, 12, 10],
        [13, 14, 11, 18, 14, 18, 13,  4, 14,  8,  1,  6,  4, 12, 19, 18,  2, 14,
          4,  8, 10, 18, 14, 19,  7,  6,  8, 10, 19, 11],
        [11,  6, 14, 11,  1,  8, 19, 14, 14, 17, 18,  7,  5,  5, 17,  8, 19,  9,
         15, 11, 17,  4, 15, 18,  1,  4,  4,  1, 10,  9],
        [18, 18,  3, 14, 18

In [12]:
# this is saying that the model outputs a 50 length vector for batch element
input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)
input_lengths

tensor([50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50])

In [13]:
# the actual target sequences have lengths between S_min and S
target_lengths = torch.randint(low=S_min, high=S, size=(N,), dtype=torch.long)
target_lengths

tensor([29, 18, 11, 29, 26, 26, 12, 24, 20, 17, 26, 22, 26, 23, 27, 10])

In [16]:
ctc_loss = nn.CTCLoss()

In [23]:
loss = ctc_loss(input, target, input_lengths, target_lengths)
loss

tensor(6.0106, grad_fn=<MeanBackward0>)

In [25]:
prob = torch.exp(-loss)
prob

tensor(0.0025, grad_fn=<ExpBackward>)

In [27]:
f_loss = 0.25 * (1 - prob)**0.5 * loss
f_loss

tensor(1.5008, grad_fn=<MulBackward0>)

In [28]:
f_loss.backward()